# Estimating Tour Mode Choice

This notebook illustrates how to re-estimate tour and subtour mode choice for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


{'larch': '6.0.41',
 'sharrow': '2.13.1.dev2+g065e1fe',
 'numpy': '1.24.4',
 'pandas': '2.3.0',
 'xarray': '2025.4.0',
 'numba': '0.60.0',
 'jax': 'not installed'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare, backup

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

In this demo notebook, we will (later) edit some model files.  But for demo purposes, we want to
make sure we are starting from the "original" files, so we'll check that now.  For actual 
applications, this step would not be necessary.

In [3]:
backup("output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_coefficients.csv")
backup("output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_coefficients_template.csv")
backup("output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_SPEC.csv")

# Load data and prep model for estimation

In [4]:
modelname = "tour_mode_choice"

from activitysim.estimation.larch import component_model

model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

loading from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_coefficients.csv
loading from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_coefficients_template.csv
loading spec from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_SPEC.csv
loading from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_values_combined.parquet


The tour mode choice model is already a `ModelGroup` segmented on different purposes,
so we can add the subtour mode choice as just another member model of the group

In [5]:
model2, data2 = component_model(
    "atwork_subtour_mode_choice", 
    edb_directory="output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice",
    return_data=True,
    )

loading from output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice/tour_mode_choice_coefficients.csv
loading from output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice/atwork_subtour_mode_choice_coefficients_template.csv
loading spec from output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice/atwork_subtour_mode_choice_SPEC.csv
loading from output-est-mode/estimation_data_bundle/atwork_subtour_mode_choice/atwork_subtour_mode_choice_values_combined.parquet


In [6]:
model.extend(model2)

# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

### Coefficients

In [7]:
data.coefficients

,value,constrain
coefficient_name,,
coef_one,1.000,T
coef_nest_root,1.000,T
coef_nest_AUTO,0.720,T
coef_nest_AUTO_DRIVEALONE,0.350,T
coef_nest_AUTO_SHAREDRIDE2,0.350,T
...,...,...
drive_transit_CBD_ASC_school_univ,0.672,F
drive_transit_CBD_ASC_work,1.100,F
drive_transit_CBD_ASC_atwork,0.564,F


#### Utility specification

In [8]:
data.spec

,Label,Description,Expression,DRIVEALONEFREE,DRIVEALONEPAY,SHARED2FREE,SHARED2PAY,SHARED3FREE,SHARED3PAY,WALK,...,WALK_HVY,WALK_COM,DRIVE_LOC,DRIVE_LRF,DRIVE_EXP,DRIVE_HVY,DRIVE_COM,TAXI,TNC_SINGLE,TNC_SHARED
0,util_DRIVEALONEFREE_Unavailable,DRIVEALONEFREE - Unavailable,sov_available == False,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,util_DRIVEALONEFREE_Unavailable_for_zero_auto_...,DRIVEALONEFREE - Unavailable for zero auto hou...,auto_ownership == 0,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,util_DRIVEALONEFREE_Unavailable_for_persons_le...,DRIVEALONEFREE - Unavailable for persons less ...,age < 16,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,util_DRIVEALONEFREE_Unavailable_for_joint_tours,DRIVEALONEFREE - Unavailable for joint tours,is_joint == True,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,util_DRIVEALONEFREE_Unavailable_if_didnt_drive...,DRIVEALONEFREE - Unavailable if didn't drive t...,is_atwork_subtour & ~work_tour_is_SOV,-999,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,util_Drive_to_Transit_distance_penalty,Drive to Transit - distance penalty,@drvtrn_distpen_0_multiplier * (1-od_skims['DI...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,coef_ivt,coef_ivt,coef_ivt,coef_ivt,coef_ivt,NaN,NaN,NaN
312,util_Walk_not_available_for_long_distances,Walk not available for long distances,@od_skims.max('DISTWALK') > 3,NaN,NaN,NaN,NaN,NaN,NaN,-999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
313,util_Bike_not_available_for_long_distances,Bike not available for long distances,@od_skims.max('DISTBIKE') > 8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
314,util_Drive_alone_not_available_for_escort_tours,Drive alone not available for escort tours,is_escort,-999,-999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Chooser data

In [9]:
data.chooser_data

,model_choice,override_choice,util_DRIVEALONEFREE_Unavailable,util_DRIVEALONEFREE_Unavailable_for_zero_auto_households,util_DRIVEALONEFREE_Unavailable_for_persons_less_than_16,util_DRIVEALONEFREE_Unavailable_for_joint_tours,util_DRIVEALONEFREE_Unavailable_if_didnt_drive_to_work,util_DRIVEALONEFREE_In_vehicle_time,util_DRIVEALONEFREE_Terminal_time,util_DRIVEALONEFREE_Operating_cost,...,walk_lrf_available,walk_ferry_available,drive_local_available,drive_commuter_available,drive_express_available,drive_heavyrail_available,drive_lrf_available,drive_ferry_available,destination_in_cbd,override_choice_code
tour_id,,,,,,,,,,,,,,,,,,,,,
1870,SHARED3FREE,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,20.830002,21.26572,6.293324,...,False,False,True,True,False,True,False,False,1,5
20468,SHARED3FREE,TNC_SINGLE,0.0,0.0,0.0,0.0,0.0,9.440000,13.24624,14.917349,...,False,False,True,True,False,True,False,False,0,20
27055,SHARED3FREE,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,13.160000,8.53068,25.088453,...,False,False,True,True,False,True,False,False,0,5
38877,SHARED2FREE,SHARED3FREE,0.0,0.0,0.0,0.0,0.0,6.730000,11.04132,11.040045,...,False,False,True,True,False,True,False,False,0,5
38904,SHARED2FREE,SHARED2FREE,0.0,0.0,0.0,0.0,0.0,22.410000,17.29320,46.920191,...,True,False,True,True,False,True,True,False,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310197956,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,29.900002,4.21092,69.245528,...,False,False,False,False,False,False,False,False,0,1
310202376,DRIVEALONEFREE,BIKE,0.0,0.0,0.0,0.0,0.0,5.480000,3.82392,19.277738,...,False,False,False,False,False,False,False,False,0,8
310202384,DRIVEALONEFREE,DRIVEALONEFREE,0.0,0.0,0.0,0.0,0.0,5.480000,3.82392,19.277738,...,False,False,False,False,False,False,False,False,0,1


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [10]:
model.doctor(repair_ch_av="-")

(<larch.ModelGroup>,
 [(<larch.Model (GEV) "eatout">, dictx()),
  (<larch.Model (GEV) "escort">, dictx()),
  (<larch.Model (GEV) "othdiscr">, dictx()),
  (<larch.Model (GEV) "othmaint">, dictx()),
  (<larch.Model (GEV) "school">, dictx()),
  (<larch.Model (GEV) "shopping">, dictx()),
  (<larch.Model (GEV) "social">, dictx()),
  (<larch.Model (GEV) "univ">, dictx()),
  (<larch.Model (GEV) "work">, dictx()),
  (<larch.Model (GEV) "atwork">, dictx())])

In [11]:
result = model.maximize_loglike(method="SLSQP", options={"maxiter": 1000})

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
-999,-999.000000,-999.000000,-999.000000,-999.0,-999.0,0.0,1
1,1.000000,1.000000,1.000000,1.0,1.0,0.0,1
bike_ASC_auto_deficient_atwork,-1.054274,-1.054274,-0.807408,-inf,inf,0.0,0
bike_ASC_auto_deficient_eatout,-1.458927,-1.458927,-1.569111,-inf,inf,0.0,0
bike_ASC_auto_deficient_escort,-3.980018,-3.980018,-4.527928,-inf,inf,0.0,0
...,...,...,...,...,...,...,...
walk_transit_ASC_no_auto_work,4.604150,4.604150,5.035417,-inf,inf,0.0,0
walk_transit_CBD_ASC_atwork,1.296785,1.296785,0.564000,-inf,inf,0.0,0
walk_transit_CBD_ASC_eatout_escort_othdiscr_othmaint_shopping_social,1.078110,1.078110,0.525000,-inf,inf,0.0,0


/Users/jpn/Git/aestival/repos/larch/src/larch/model/optimization.py:338: UserWarning: SLSQP may not play nicely with unbounded parameters
if you get poor results, consider setting global bounds with model.set_cap()
  warnings.warn(  # infinite bounds #  )


In [12]:
model.calculate_parameter_covariance()

/var/folders/gp/zzfchnm91870k7pvbkmbgwz80000gr/T/ipykernel_45605/3375301916.py:1: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  model.calculate_parameter_covariance()


(array([0.00000000e+00, 0.00000000e+00, 3.43568882e-01, 3.95622877e-01,
        4.06484604e-01, 2.03367924e-01, 3.22743381e-01, 9.64080406e-01,
        2.18326807e-01, 3.24554296e-01,            nan, 9.25561766e-02,
        4.69818109e+02, 1.59418569e-01, 3.37340862e-01, 1.05444367e-01,
        1.93207745e-01, 9.18689416e-01, 1.38355178e-01, 2.06922312e-01,
        6.56527431e-03, 8.14434243e-02, 7.65331643e+01, 7.59009053e+01,
        7.59050227e+01, 7.59000796e+01, 7.58995284e+01, 1.08570448e+02,
        7.58994612e+01, 7.58998820e+01, 1.48396628e-03, 7.58997989e+01,
        7.84724197e-03, 1.75813808e-01, 8.99726958e-02, 2.05380801e-01,
        4.25371868e-02, 7.47682204e-02, 8.42243554e-02, 6.82331737e-02,
        3.12375884e-02, 5.71491736e-02, 1.78428279e-02, 9.12338677e-02,
        4.78568573e-04, 1.44741724e-04, 3.15779236e-04, 1.47478876e-04,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

### Estimated coefficients

In [13]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value
Parameter,,,,,
-999,-999.,0.00,NA,,0.00
1,1.00,0.00,NA,,0.00
bike_ASC_auto_deficient_atwork,-1.05,0.344,-3.07,**,0.00
bike_ASC_auto_deficient_eatout,-1.46,0.396,-3.69,***,0.00
bike_ASC_auto_deficient_escort,-3.98,0.406,-9.79,***,0.00
bike_ASC_auto_deficient_othdiscr,-0.910,0.203,-4.47,***,0.00
bike_ASC_auto_deficient_othmaint,-2.06,0.323,-6.37,***,0.00
bike_ASC_auto_deficient_school,-1.20,0.964,-1.24,,0.00
bike_ASC_auto_deficient_shopping,-1.74,0.218,-7.99,***,0.00


# Output Estimation Results

In [14]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [15]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [16]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_one,1.000000,T
1,coef_nest_root,1.000000,T
2,coef_nest_AUTO,0.720000,T
3,coef_nest_AUTO_DRIVEALONE,0.350000,T
4,coef_nest_AUTO_SHAREDRIDE2,0.350000,T
...,...,...,...
304,drive_transit_CBD_ASC_school_univ,1.227148,F
305,drive_transit_CBD_ASC_work,1.636347,F
306,drive_transit_CBD_ASC_atwork,0.564000,F
307,coef_test_eatout_escort_othdiscr_othmaint_shop...,0.076990,F


# Modify Spec

Here, we will demonstrate the process of re-estimating the model with a modified
SPEC file.  This does *not* require re-running ActivitySim, it just requires
changing the SPEC file, and possibly the coefficients and/or coefficients template files, and re-running the Larch estimation only.

The `backup` command we ran earlier made a backup copy of the
original spec file in the EDB directory.
This was not strictly necessary, but since we're about to modify it and
we may want undo our changes, it can be handy to keep a copy of the
original spec file around. Since we already have a backup copy, we'll make some 
changes directly in the SPEC file.  As an example here, we're going
to add a constant term to the model, with a coefficient that is shared in common across 
several (but not all) purposes.

For this demo we are editing 
the model files using Python code to make the changes, but a user does not need
to change the file using Python; any CSV editor (e.g. Excel) can be used. We will
edit three files: the spec, the coefficients, and the coefficients template.
We're not going to edit any of the existing content in any of these files, instead 
we will simply add a few lines to each to add an extra expression to the spec, and
the coefficients for that expression.

In [17]:
with open(data.edb_directory / "tour_mode_choice_SPEC.csv", mode="a") as f:
    f.write("util_test,Additional DA constant,1,coef_test,,,,,,,,,,,,,,,,,,,,\n")

In [18]:
with open(data.edb_directory / "tour_mode_choice_coefficients_template.csv", mode="a") as f:
    f.write("coef_test,"
            "coef_test_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork,"
            "coef_test_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork,"
            "coef_test_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork,"
            "coef_test_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork,"
            "coef_test_school_univ,"
            "coef_test_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork,"
            "coef_test_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork,"
            "coef_test_school_univ,"
            "coef_test_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork,"
            "coef_test_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork\n")

In [19]:
with open(data.edb_directory / "tour_mode_choice_coefficients.csv", mode="a") as f:
    f.write("coef_test_eatout_escort_othdiscr_othmaint_shopping_social_work_atwork,0,F\n"
            "coef_test_school_univ,0,F\n")

Now to re-estimate the model, we just re-run the same steps as the original estimation above.

In [21]:
remodel, redata = component_model(modelname, edb_directory=str(data.edb_directory), return_data=True)

loading from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_coefficients.csv
loading from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_coefficients_template.csv
loading spec from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_SPEC.csv
loading from output-est-mode/estimation_data_bundle/tour_mode_choice/tour_mode_choice_values_combined.parquet


In [22]:
remodel.extend(model2)

In [23]:
remodel.estimate(maxiter=200)

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
-999,-999.000000,-999.000000,-999.000000,-999.0,-999.0,0.0,1
1,1.000000,1.000000,1.000000,1.0,1.0,0.0,1
bike_ASC_auto_deficient_atwork,-1.059385,-1.059385,-0.807408,-inf,inf,0.0,0
bike_ASC_auto_deficient_eatout,-1.201250,-1.201250,-1.569111,-inf,inf,0.0,0
bike_ASC_auto_deficient_escort,-3.880330,-3.880330,-4.527928,-inf,inf,0.0,0
...,...,...,...,...,...,...,...
walk_transit_ASC_no_auto_work,4.631023,4.631023,5.035417,-inf,inf,0.0,0
walk_transit_CBD_ASC_atwork,1.184225,1.184225,0.564000,-inf,inf,0.0,0
walk_transit_CBD_ASC_eatout_escort_othdiscr_othmaint_shopping_social,1.088968,1.088968,0.525000,-inf,inf,0.0,0


/Users/jpn/Git/aestival/repos/larch/src/larch/model/optimization.py:338: UserWarning: slsqp may not play nicely with unbounded parameters
if you get poor results, consider setting global bounds with model.set_cap()
  warnings.warn(  # infinite bounds #  )
/Users/jpn/Git/aestival/repos/larch/src/larch/model/model_group.py:366: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  self.calculate_parameter_covariance()


┣                x: -999                                                                   -999.000000
┃                   1                                                                         1.000000
┃                   bike_ASC_auto_deficient_atwork                                           -1.059385
┃                   bike_ASC_auto_deficient_eatout                                           -1.201250
┃                   bike_ASC_auto_deficient_escort                                           -3.880330
┃                                                                                              ...    
┃                   walk_transit_ASC_no_auto_work                                             4.631023
┃                   walk_transit_CBD_ASC_atwork                                               1.184225
┃                   walk_transit_CBD_ASC_eatout_escort_othdiscr_othmaint_shopping_social      1.088968
┃                   walk_transit_CBD_ASC_school_univ                                          0.826484
┃                   walk_transit_CBD_ASC_work                                                 1.146513
┃                   Length: 312, dtype: float64
┣          logloss: 1.0933400042471608
┣        d_logloss: -999                                                                    0.000000
┃                   1                                                                       0.000000
┃                   bike_ASC_auto_deficient_atwork                                          0.000018
┃                   bike_ASC_auto_deficient_eatout                                          0.000005
┃                   bike_ASC_auto_deficient_escort                                          0.000010
┃                                                                                             ...   
┃                   walk_transit_ASC_no_auto_work                                           0.000049
┃                   walk_transit_CBD_ASC_atwork                                            -0.000097
┃                   walk_transit_CBD_ASC_eatout_escort_othdiscr_othmaint_shopping_social    0.000019
┃                   walk_transit_CBD_ASC_school_univ                                        0.000020
┃                   walk_transit_CBD_ASC_work                                               0.000038
┃                   Length: 312, dtype: float64
┣              nit: 200
┣             nfev: 211
┣             njev: 200
┣           status: 9
┣          message: 'Iteration limit reached'
┣          success: False
┣     elapsed_time: datetime.timedelta(seconds=106, microseconds=414845)
┣           method: 'slsqp'
┣          n_cases: 69971
┣ iteration_number: 200
┣          loglike: -76502.09343717809

We can easily review the parameter estimates from the original and
revised models side by side to see what changed.

In [24]:
with pd.option_context('display.max_rows', 999):
    display(pd.concat({
        "model": model.parameter_summary().data,
        "remodel": remodel.parameter_summary().data,
    }, axis=1).fillna(""))

model             \
                                                        Value    Std Err   
Parameter                                                                  
-999                                                    -999.       0.00   
1                                                        1.00       0.00   
bike_ASC_auto_deficient_atwork                          -1.05      0.344   
bike_ASC_auto_deficient_eatout                          -1.46      0.396   
bike_ASC_auto_deficient_escort                          -3.98      0.406   
bike_ASC_auto_deficient_othdiscr                       -0.910      0.203   
bike_ASC_auto_deficient_othmaint                        -2.06      0.323   
bike_ASC_auto_deficient_school                          -1.20      0.964   
bike_ASC_auto_deficient_shopping                        -1.74      0.218   
bike_ASC_auto_deficient_social                          0.361      0.325   
bike_ASC_auto_deficient_univ                           -0.669         NA   
bike_ASC_auto_deficient_work                           -0.279     0.0926   
bike_ASC_auto_sufficient_atwork                          15.7       470.   
bike_ASC_auto_sufficient_eatout                         -1.85      0.159   
bike_ASC_auto_sufficient_escort                         -5.65      0.337   
bike_ASC_auto_sufficient_othdiscr                       -1.86      0.105   
bike_ASC_auto_sufficient_othmaint                       -3.17      0.193   
bike_ASC_auto_sufficient_school                         -2.21      0.919   
bike_ASC_auto_sufficient_shopping                       -3.14      0.138   
bike_ASC_auto_sufficient_social                         -2.14      0.207   
bike_ASC_auto_sufficient_univ                           -1.94    0.00657   
bike_ASC_auto_sufficient_work                           -1.93     0.0814   
bike_ASC_no_auto_atwork                                -0.910       76.5   
bike_ASC_no_auto_eatout                                 0.428       75.9   
bike_ASC_no_auto_escort                                -0.745       75.9   
bike_ASC_no_auto_othdiscr                              -0.501       75.9   
bike_ASC_no_auto_othmaint                                1.53       75.9   
bike_ASC_no_auto_school                                  13.5       109.   
bike_ASC_no_auto_shopping                               0.951       75.9   
bike_ASC_no_auto_social                                 0.374       75.9   
bike_ASC_no_auto_univ                                    4.29    0.00148   
bike_ASC_no_auto_work                                    3.43       75.9   
coef_age010_trn_multiplier_atwork                    0.000722    0.00785   
coef_age010_trn_multiplier_eatout_escort_othdis...      0.490      0.176   
coef_age010_trn_multiplier_school_univ                 -0.851     0.0900   
coef_age1619_da_multiplier_atwork                      -0.175      0.205   
coef_age1619_da_multiplier_eatout_escort_othdis...     0.0357     0.0425   
coef_age1619_da_multiplier_school_univ                  -1.67     0.0748   
coef_age16p_sr_multiplier_eatout_escort_othdisc...      -1.58     0.0842   
coef_age16p_sr_multiplier_school_univ_work_atwork      -0.688     0.0682   
coef_hhsize1_sr_multiplier_eatout_escort_othdis...    -0.0721     0.0312   
coef_hhsize1_sr_multiplier_work                        -0.822     0.0571   
coef_hhsize2_sr_multiplier_eatout_escort_othdis...   -0.00404     0.0178   
coef_hhsize2_sr_multiplier_school_univ                 -0.743     0.0912   
coef_ivt_atwork                                       -0.0112   0.000479   
coef_ivt_eatout_escort_othdiscr_othmaint_shoppi...   -0.00691   0.000145   
coef_ivt_school_univ                                  -0.0103   0.000316   
coef_ivt_work                                        -0.00791   0.000147   
coef_nest_AUTO                                          0.720       0.00   
coef_nest_AUTO_DRIVEALONE                               0.350       0.00   
coef_nest_AUTO_SHAREDRIDE2  

In [25]:
with pd.option_context('display.max_rows', 999):
    display(pd.concat({
        "model": model.estimation_statistics_raw(),
        "remodel": remodel.estimation_statistics_raw(),
    }, axis=1).fillna(""))

model        remodel
Number of Cases                    Aggregate   69971.000000   69971.000000
Log Likelihood at Convergence      Aggregate  -76508.289239  -76502.093437
                                   Per Case       -1.093429      -1.093340
Log Likelihood at Null Parameters  Aggregate -142469.530288 -142469.530288
                                   Per Case       -2.036123      -2.036123
Rho Squared w.r.t. Null Parameters Aggregate       0.462985       0.463028